In [1]:
import os
from xml.dom import minidom
import Wiki2Text

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import SimpleStatement

In [2]:
current_path = os.getcwd()

In [3]:
xml_file = "first_100k.xml"

In [4]:
server = ['hadoop05.f4.htw-berlin.de']
port = '9042' 
login = '' # hier dein cassandra login
pw = '' # hier dein cassandra password

keyspace = 'wiki_word2vec'

In [5]:
auth_provider = PlainTextAuthProvider(username=login, password=pw)

In [6]:
cluster = Cluster(contact_points=server, port=port, auth_provider=auth_provider)
session = cluster.connect(keyspace)

In [7]:
xmldoc = minidom.parse(current_path + '/' + xml_file)
pageList = xmldoc.getElementsByTagName('page')
len(pageList)

385

In [8]:
id_number = 1

def getNewId():
    result = id_number
    id_number += 1
    return result

In [9]:
table = 'id_title_sentence'

session.execute("""
        CREATE TABLE IF NOT EXISTS """+ table +""" (
            id UUID PRIMARY KEY,
            page_id INT,
            page_title text,
            page_sentence text
        )
        """)

for page in pageList:
    page_id = page.getElementsByTagName("id")[0].firstChild.nodeValue
    page_title = page.getElementsByTagName("title")[0].firstChild.nodeValue
    
    text = page.getElementsByTagName("text")[0].firstChild.nodeValue
    page_text = Wiki2Text.getOriginalSentences(text)
    
    text_list = Wiki2Text.getSentencesFromText(page_text)
    
    for sentence in text_list:
        satz = ''.join(sentence)
        myQuery = "INSERT INTO "+ table +" (id, page_id, page_title, page_sentence) VALUES (now(),"+page_id+",'"+page_title+"', '"+satz+"');"
        try:
            session.execute(myQuery)
        except:
            pass
    